In [6]:
import sys
sys.path.append('C:\\PYTHONprojects\\en')
sys.path.append('C:\\PYTHONprojects')
sys.path.append('C:\\PYTHONprojects\\utilities')
import morePVs_output as opm
import morePVs as mpv
import os
import en_utilities as um
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.lines as mlines
import shutil
import seaborn as sns

Peak Demand wrt kWp and kWh
----------------------------------------------
1) AnnualPeak Load:
* plot peak load vs kWp / kWh (average across all VBs)
1a) Box plotof peak to show dist across VBS
*

2) Frequency Distribution:
* Plot frequency curve of daily peak or timestamp peak

3) Ten peaks
* As 1) but plot average of top 10 (?) peaks



 


In [7]:
sitelist = ['F', 'G','H','I','J']

arrangement = 'en_pv'
fontsize=12
#sitelist=['J']

In [8]:
# Run this cell only once to collate peak demand data from timeseries files
for sitename in sitelist:             
    # -------------------------------
    # combine  input and result files
    # -------------------------------
    project='EN2_bat2'
    study_name='bat_energy3_' + sitename
    base_path='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_4\\studies'
    op = opm.Output(base_path = base_path,
                              project = project,
                              study_name = study_name)
    self = op
    df = self.data.copy()
    df_in = self.study_parameters.copy()
    df = df.merge(df_in,left_index = True, right_index=True)
    for c in df.columns:
        if '_x' in c:
            nc = c[:-2]
            df.rename(columns={c:nc}, inplace=True)
        if '_y' in c:
            df= df.drop([c],axis=1)
    
    # ---------------
    # Organise labels
    # ---------------
    df.loc[:,'site'] = df.loc[:,'load_folder'].apply(lambda x : x[-1])
#     df.loc[:, 'labels'] = df.loc[:,'site']
#     floors = {'A':12,'E':7, 'D':9,'B':8,'H':3,'I':4,'G':4,'C':4,'J':4,'F':5}
#     numhouses={'A':208,'E':161, 'D':138,'B':104,'H':52,'I':48,'G':44,'C':34,'J':26,'F':20}
#     labels ={}
#     for s, f in floors.items():
#         u = numhouses[s]
#         labels[s] = s +'('+str(u)+'/'+str(f)+')'
#     # print(labels)
#     df.loc[:, 'label'] = df.loc[:, 'site'].apply(lambda x: labels[x])
#     # ----------------
#     # Create Site tags
#     # ----------------
#     site_tag ={'A': 'a208_f12_cp34',
#      'B': 'a104_f8_cp57',
#      'C': 'a34_f4_cp33',
#      'D': 'a138_f9_cp44',
#      'E': 'a161_f7_cp38',
#      'F': 'a20_f5_cp36',
#      'G': 'a44_f4_cp17',
#      'H': 'a52_f3_cp26',
#      'I': 'a48_f4_cp09',
#      'J': 'a26_f4_cp43'}

  # get pv sizes for maximum pv systems
    # -----------------------------------
    maximums={}
    pv_ref_file='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_3\\reference\\capex_pv_lookup.csv'
    pv_ref=pd.read_csv(pv_ref_file)
    pv_ref = pv_ref.set_index('pv_cap_id')
    for i in pv_ref.index:
        if 'max' in i and 'site' in i:
            site = um.find_between(i,'_','_')
            maximums[site] = pv_ref.loc[i,'kW']

    # get kWp / unit ,kWh / unit  and % bau for each scenario
    # ------------------------------------------------------------
    df['pv_filename'].fillna('zero',inplace=True)

    for s in df.index:
        site = df.loc[s,'load_folder'][-1]
        df.loc[s,'site']=site
        if df.loc[s,'pv_filename'] == 'zero':
            df.loc[s,'kWp_unit'] =0
        elif 'max' in df.loc[s,'pv_filename']:
            df.loc[s,'kWp_unit']= maximums[site]/df.loc[s,'number_of_households']
        else:
            df.loc[s,'kWp_unit'] = float(df.loc[s,'pv_filename'][-9]) + float(df.loc[s,'pv_filename'][-7])/10

        df.loc[s,'kWh_unit']= df.fillna(0).loc[s,'central_battery_capacity_kWh']/df.loc[s,'number_of_households']
        df.loc[s,'short_label']= df.loc[s,'scenario_label'][14:24]

    # get timeseries data and create dfs with peak demand
    # -----------------------------------------
    ts_path = os.path.join(base_path,project,'outputs',study_name,'timeseries')
    tslist = os.listdir(ts_path)

    process_path = os.path.join(base_path,project,'outputs',study_name,'process')
    if not os.path.exists(process_path):
        os.makedirs(process_path)

    # paths for peak demand files  
    pf = 'peak_demand.csv'
    peakFile = os.path.join(process_path, pf)
    pf10 = 'peak_demand_10.csv'
    peakFile10 = os.path.join(process_path, pf10)

    # Calc single higest demand:
    dfpeak = pd.DataFrame(index = df['short_label'], columns =[str(r).zfill(2) for r in range(1,51)])
    for tsf in tslist:
        short_label = tsf[0:10]
        vb = tsf[-6:-4]
        tsFile = os.path.join(ts_path, tsf)
        dfts = pd.read_csv(tsFile)        
        dfpeak.loc[short_label,vb] = dfts['grid_import'].max()
        
    um.df_to_csv(dfpeak,peakFile)
   

    # Calc mean of 10 higest demands:
    dfpeak10 = pd.DataFrame(index = df['short_label'], columns =[str(r).zfill(2) for r in range(1,51)])    
    for tsf in tslist:
        short_label = tsf[0:10]
        vb = tsf[-6:-4]
        tsFile = os.path.join(ts_path, tsf)
        dfts = pd.read_csv(tsFile)
        dfpeak10.loc[short_label,vb] = dfts['grid_import'].nlargest(10).mean()
    um.df_to_csv(dfpeak10,peakFile10)


KeyboardInterrupt: 

In [ ]:
#sitelist = ['J','I', 'H', 'G', 'F']
# # Single battery strategy:
# battery_strategies = ['evening_discharge_1']

# for sitename in sitelist:             
#     # -------------------------------
#     # combine  input and result files
#     # -------------------------------

#     project='EN2_bat2'
#     study_name='energy1_' + sitename
#     base_path='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_4\\studies'
#     op = opm.Output(base_path = base_path,
#                               project = project,
#                               study_name = study_name)
#     self = op
#     df = self.data.copy()
#     df_in = self.study_parameters.copy()
#     df = df.merge(df_in,left_index = True, right_index=True)
#     for c in df.columns:
#         if '_x' in c:
#             nc = c[:-2]
#             df.rename(columns={c:nc}, inplace=True)
#         if '_y' in c:
#             df= df.drop([c],axis=1)
#     df.head()
#     arrangement = 'en_pv'
#     fontsize=12

    
#     # ---------------
#     # Organise labels
#     # ---------------
#     df.loc[:,'site'] = df.loc[:,'load_folder'].apply(lambda x : x[-1])
#     df.loc[:, 'labels'] = df.loc[:,'site']
#     floors = {'A':12,'E':7, 'D':9,'B':8,'H':3,'I':4,'G':4,'C':4,'J':4,'F':5}
#     numhouses={'A':208,'E':161, 'D':138,'B':104,'H':52,'I':48,'G':44,'C':34,'J':26,'F':20}
#     labels ={}
#     for s, f in floors.items():
#         u = numhouses[s]
#         labels[s] = s +'('+str(u)+'/'+str(f)+')'
#     # print(labels)
#     df.loc[:, 'label'] = df.loc[:, 'site'].apply(lambda x: labels[x])
#     # ----------------
#     # Create Site tags
#     # ----------------
#     site_tag ={'A': 'a208_f12_cp34',
#      'B': 'a104_f8_cp57',
#      'C': 'a34_f4_cp33',
#      'D': 'a138_f9_cp44',
#      'E': 'a161_f7_cp38',
#      'F': 'a20_f5_cp36',
#      'G': 'a44_f4_cp17',
#      'H': 'a52_f3_cp26',
#      'I': 'a48_f4_cp09',
#      'J': 'a26_f4_cp43'}

#    # get pv sizes for maximum pv systems
#     # -----------------------------------
#     maximums={}
#     pv_ref_file='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_3\\reference\\capex_pv_lookup.csv'
#     pv_ref=pd.read_csv(pv_ref_file)
#     pv_ref = pv_ref.set_index('pv_cap_id')
#     for i in pv_ref.index:
#         if 'max' in i and 'site' in i:
#             site = um.find_between(i,'_','_')
#             maximums[site] = pv_ref.loc[i,'kW']

#     # get kWp / unit ,kWh / unit  and % bau for each scenario
#     # ------------------------------------------------------------
#     df['pv_filename'].fillna('zero',inplace=True)

#     for s in df.index:
#         site = df.loc[s,'load_folder'][-1]
#         df.loc[s,'site']=site
#         if df.loc[s,'pv_filename'] == 'zero':
#             df.loc[s,'kWp_unit'] =0
#         elif 'max' in df.loc[s,'pv_filename']:
#             df.loc[s,'kWp_unit']= maximums[site]/df.loc[s,'number_of_households']
#         else:
#             df.loc[s,'kWp_unit'] = float(df.loc[s,'pv_filename'][-9]) + float(df.loc[s,'pv_filename'][-7])/10

#         df.loc[s,'kWh_unit']= df.fillna(0).loc[s,'central_battery_capacity_kWh']/df.loc[s,'number_of_households']
#         df.loc[s,'short_label']= df.loc[s,'scenario_label'][12:24]
    




In [9]:
site_tag ={'A': 'a208_f12_cp34',
 'B': 'a104_f8_cp57',
 'C': 'a34_f4_cp33',
 'D': 'a138_f9_cp44',
 'E': 'a161_f7_cp38',
 'F': 'a20_f5_cp36',
 'G': 'a44_f4_cp17',
 'H': 'a52_f3_cp26',
 'I': 'a48_f4_cp09',
 'J': 'a26_f4_cp43'}
sitelist = ['F','G','H','I','J']

In [19]:
yrange ={ 'F': [.87,.895],
 'G': [.685,.71],
 'H': [.675,.705],
 'I': [.665,.68],
 'J': [.865,.905]}

In [10]:
# #This cell plots boxplots of peak demand vs 
# # Single battery strategy:
# battery_strategies = ['evening_discharge_1']
# for sitename in sitelist:             
#     # -------------------------------
#     # combine  input and result files
#     # -------------------------------

#     project='EN2_bat2'
#     study_name='bat_energy2_' + sitename
#     base_path='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_4\\studies'
#     op = opm.Output(base_path = base_path,
#                               project = project,
#                               study_name = study_name)
#     self = op
#     df = self.data.copy()
#     df_in = self.study_parameters.copy()
#     df = df.merge(df_in,left_index = True, right_index=True)
#     for c in df.columns:
#         if '_x' in c:
#             nc = c[:-2]
#             df.rename(columns={c:nc}, inplace=True)
#         if '_y' in c:
#             df= df.drop([c],axis=1)
#     df.head()
#     arrangement = 'en_pv'
#     fontsize=12
    
#     # get pv sizes for maximum pv systems
#     # -----------------------------------
#     maximums={}
#     pv_ref_file='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_3\\reference\\capex_pv_lookup.csv'
#     pv_ref=pd.read_csv(pv_ref_file)
#     pv_ref = pv_ref.set_index('pv_cap_id')
#     for i in pv_ref.index:
#         if 'max' in i and 'site' in i:
#             site = um.find_between(i,'_','_')
#             maximums[site] = pv_ref.loc[i,'kW']

#     # get kWp / unit ,kWh / unit  and % bau for each scenario
#     # ------------------------------------------------------------
#     df['pv_filename'].fillna('zero',inplace=True)

#     for s in df.index:
#         site = df.loc[s,'load_folder'][-1]
#         df.loc[s,'site']=site
#         if df.loc[s,'pv_filename'] == 'zero':
#             df.loc[s,'kWp_unit'] =0
#         elif 'max' in df.loc[s,'pv_filename']:
#             df.loc[s,'kWp_unit']= maximums[site]/df.loc[s,'number_of_households']
#         else:
#             df.loc[s,'kWp_unit'] = float(df.loc[s,'pv_filename'][-9]) + float(df.loc[s,'pv_filename'][-7])/10

#         df.loc[s,'kWh_unit']= df.fillna(0).loc[s,'central_battery_capacity_kWh']/df.loc[s,'number_of_households']
#         df.loc[s,'short_label']= df.loc[s,'scenario_label'][14:24]
    

#     # paths for peak demand files  
#     # ---------------------------
#     process_path = os.path.join(base_path,project,'outputs',study_name,'process')
#     pf = 'peak_demand.csv'
#     peakFile = os.path.join(process_path, pf)
#     pf10 = 'peak_demand_10.csv'
#     peakFile10 = os.path.join(process_path, pf10)

#     #  reload files here if saved previously:
#     dfpeak = pd.read_csv(peakFile)
#     dfpeak=dfpeak.set_index('short_label')
#     dfpeak10 = pd.read_csv(peakFile10)
#     dfpeak10=dfpeak10.set_index('short_label')

#     df['mean_peak']= df['short_label'].apply(lambda x:dfpeak.loc[x].mean())
#     df['mean_peak_10highest']= df['short_label'].apply(lambda x:dfpeak10.loc[x].mean())


    
#     # Directories for plots
#     # ---------------------
#     plot_pd = os.path.join(op.plot_path,'peak_demand')
#     if not os.path.exists(plot_pd):
#         os.makedirs(plot_pd)
#     plot_pd10 = os.path.join(op.plot_path,'peak_demand_10highest')
#     if not os.path.exists(plot_pd10):
#         os.makedirs(plot_pd10)

#     plot_pf = os.path.join(op.plot_path,'peak_frequency')
#     if not os.path.exists(plot_pf):
#         os.makedirs(plot_pf)

#     box_path = os.path.join(op.plot_path,'peak_box')
#     if not os.path.exists(box_path):
#             os.makedirs(box_path)


#     # ONE AY (1a)
#     # BoxPlot peak across all VBs 
#     # ----------------------------

#     # Use seaborn - boxplot:
#     sns.set(style="whitegrid")
#     cmap = mpl.cm.tab10_r
#     colours = [cmap(k) for k in [1,2,3,4,5,6,7]]  
#     colourp = sns.color_palette(palette=colours)

#     for strategy in battery_strategies:      
#         # --------------------------------------
#         dfen = df[df['arrangement'].isin(['en', 'en_pv'])]
#         dfx=dfen.copy().loc[dfen['central_battery_strategy']==strategy].append(
#             dfen.copy().loc[dfen['kWh_unit']==0]).append(
#             df[df['arrangement'] =='bau'])
#         # Create df for boxplots:
#         dfo = pd.DataFrame(columns=[ 'kWp_unit','kWh_unit', 'peak_demand'])

#         for s in dfx.index:
#             kWp = '{0:3.1f}'.format(dfx.loc[s,'kWp_unit']) + ' kWp / unit'
#             kWh = dfx.loc[s,'kWh_unit']
#             for vb in range (1,51):
#                 short_label = dfx.loc[s,'short_label']
#                 peak_demand = dfpeak.loc[short_label, str(vb).zfill(2)]
#                 i = str(s) + '_' + str(vb)
#                 dfo.loc[i]= [kWp, kWh, peak_demand]


#         # Sort by kWp
#         dfo['pv_cat']=pd.Categorical(dfo['kWp_unit'],
#                                   categories = ['0.0 kWp / unit',
#                                                 '0.5 kWp / unit',
#                                                 '1.0 kWp / unit',
#                                                 '1.5 kWp / unit',
#                                                 '2.0 kWp / unit',
#                                                 '2.5 kWp / unit',
#                                                 '3.0 kWp / unit'],                                         
#                                   ordered=True)
#         dfo.sort_values(['pv_cat'], inplace=True)


#         # Plot Boxplot:
#         ax = sns.boxplot(data=dfo, x = 'kWh_unit', y= 'peak_demand', hue ='kWp_unit', 
#                          dodge=True, linewidth = 1, width=0.7,
#                         fliersize =3, notch = False,
#                         palette = colourp)
#         ax.set_xlabel("Battery Capacity (kWh / unit)", fontsize=14)
#         ax.set_ylabel("Peak Demand (kWh / 30 minutes)", fontsize=14)
#         ax.set_title('Peak demand', fontsize=20, y=1.02)
#         ax.set_ylim(15,30)
#         #Legends outside box and resize chart to fit
#         box = ax.get_position()
#         ax.set_position([box.x0, box.y0, box.width * 0.7, box.height])
#         plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
#         ax.text(1.05,0.3, site_tag[site], fontsize=14,transform=ax.transAxes)
#         ax.text(0.1,0.1, strategy, fontsize=12,transform=ax.transAxes)


#         plotFile= os.path.join(box_path, site+'_peak_demand_'+strategy+'.jpg')
#         plt.savefig(plotFile,dpi=1000)
#         plt.close()

    
#     # ONE AY (1a)
#     # repeat for average of top 10 peaks
#     # BoxPlot peak across  top 10 peaks vs kWh
#     # -----------------------------------------

#     # Use seaborn - boxplot:
#     sns.set(style="whitegrid")
#     cmap = mpl.cm.tab10_r
#     colours = [cmap(k) for k in [1,2,3,4,5,6,7]]  
#     colourp = sns.color_palette(palette=colours)


#     for strategy in battery_strategies:    

#         # --------------------------------------
#         dfen = df[df['arrangement'].isin(['en', 'en_pv'])]
#         dfx=dfen.copy().loc[dfen['central_battery_strategy']==strategy].append(
#             dfen.copy().loc[dfen['kWh_unit']==0]).append(
#             df[df['arrangement'] =='bau'])
#         # Create df for boxplots:
#         dfo = pd.DataFrame(columns=[ 'kWp_unit','kWh_unit', 'peak_demand'])

#         for s in dfx.index:
#             kWp = '{0:3.1f}'.format(dfx.loc[s,'kWp_unit']) + ' kWp / unit'
#             kWh = dfx.loc[s,'kWh_unit']
#             for vb in range (1,51):
#                 short_label = dfx.loc[s,'short_label']
#                 peak_demand = dfpeak10.loc[short_label, str(vb).zfill(2)]
#                 i = str(s) + '_' + str(vb)
#                 dfo.loc[i]= [kWp, kWh, peak_demand]


#         # Sort by kWp
#         dfo['pv_cat']=pd.Categorical(dfo['kWp_unit'],
#                                   categories = ['0.0 kWp / unit',
#                                                 '0.5 kWp / unit',
#                                                 '1.0 kWp / unit',
#                                                 '1.5 kWp / unit',
#                                                 '2.0 kWp / unit',
#                                                 '2.5 kWp / unit',
#                                                 '3.0 kWp / unit'],                                         
#                                   ordered=True)
#         dfo.sort_values(['pv_cat'], inplace=True)


#         # Plot Boxplot:
#         ax = sns.boxplot(data=dfo, x = 'kWh_unit', y= 'peak_demand', hue ='kWp_unit', 
#                          dodge=True, linewidth = 1, width=0.7,
#                         fliersize =3, notch = False,
#                         palette = colourp)
#         ax.set_xlabel("Battery Capacity (kWh / unit)", fontsize=14)
#         ax.set_ylabel("Peak Demand (kWh / 30 minutes)", fontsize=14)
#         ax.set_title('Mean of 10 demand peaks', fontsize=20, y=1.02)
#         ax.set_ylim(15,30)
#         #Legends outside box and resize chart to fit
#         box = ax.get_position()
#         ax.set_position([box.x0, box.y0, box.width * 0.7, box.height])
#         plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
#         ax.text(1.05,0.3, site_tag[site], fontsize=14,transform=ax.transAxes)
#         ax.text(0.1,0.1, strategy, fontsize=12,transform=ax.transAxes)

#         plotFile= os.path.join(box_path, site+'_peak10_demand_'+strategy+'.jpg')
#         plt.savefig(plotFile,dpi=1000)
#         plt.close()




In [11]:
# # Directories for plots
# # ---------------------
# plot_pd = os.path.join(op.plot_path,'peak_demand')
# if not os.path.exists(plot_pd):
#     os.makedirs(plot_pd)
# plot_pd10 = os.path.join(op.plot_path,'peak_demand_10highest')
# if not os.path.exists(plot_pd10):
#     os.makedirs(plot_pd10)

# plot_pd50 = os.path.join(op.plot_path,'peak_demand_50highest')
# if not os.path.exists(plot_pd50):
#     os.makedirs(plot_pd50)

# plot_pf = os.path.join(op.plot_path,'peak_frequency')
# if not os.path.exists(plot_pf):
#     os.makedirs(plot_pf)

# box_path = os.path.join(op.plot_path,'peak_box')
# if not os.path.exists(box_path):
#         os.makedirs(box_path)


# # ONE AY (1a)
# # BoxPlot peak across all VBs or top 10 peaks vs kWh
# # --------------------------------------------------

# # Use seaborn - boxplot:
# sns.set(style="whitegrid")
# cmap = mpl.cm.tab10_r
# colours = [cmap(k) for k in [1,2,3,4,5,6,7]]  #[0,0.5,1,1.5,2,2.5,3 ]]
# colourp = sns.color_palette(palette=colours)

# for strategy in battery_strategies:      
#     # --------------------------------------
#     dfen = df[df['arrangement'].isin(['en', 'en_pv'])]
#     dfx=dfen.copy().loc[dfen['central_battery_strategy']==strategy].append(
#         dfen.copy().loc[dfen['kWh_unit']==0]).append(
#         df[df['arrangement'] =='bau'])
#     # Create df for boxplots:
#     dfo = pd.DataFrame(columns=[ 'kWp_unit','kWh_unit', 'peak_demand'])

#     for s in dfx.index:
#         kWp = '{0:3.1f}'.format(dfx.loc[s,'kWp_unit']) + ' kWp / unit'
#         kWh = dfx.loc[s,'kWh_unit']
#         for vb in range (1,51):
#             short_label = dfx.loc[s,'short_label']
#             peak_demand = dfpeak.loc[short_label, str(vb).zfill(2)]
#             i = str(s) + '_' + str(vb)
#             dfo.loc[i]= [kWp, kWh, peak_demand]
  
    
#     # Sort by kWp
#     dfo['pv_cat']=pd.Categorical(dfo['kWp_unit'],
#                               categories = ['0.0 kWp / unit',
#                                             '0.5 kWp / unit',
#                                             '1.0 kWp / unit',
#                                             '1.5 kWp / unit',
#                                             '2.0 kWp / unit',
#                                             '2.5 kWp / unit',
#                                             '3.0 kWp / unit'],                                         
#                               ordered=True)
#     dfo.sort_values(['pv_cat'], inplace=True)
    

#     # Plot Boxplot:
#     ax = sns.boxplot(data=dfo, x = 'kWh_unit', y= 'peak_demand', hue ='kWp_unit', 
#                      dodge=True, linewidth = 1, width=0.7,
#                     fliersize =3, notch = False,
#                     palette = colourp)
#     ax.set_xlabel("Battery Capacity (kWh / unit)", fontsize=14)
#     ax.set_ylabel("Peak Demand (kWh / 30 minutes)", fontsize=14)
#     ax.set_title('Peak demand', fontsize=20, y=1.02)
#     ax.set_ylim(15,30)
#     #Legends outside box and resize chart to fit
#     box = ax.get_position()
#     ax.set_position([box.x0, box.y0, box.width * 0.7, box.height])
#     plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
#     ax.text(1.05,0.3, site_tag[site], fontsize=14,transform=ax.transAxes)
#     ax.text(0.1,0.1, strategy, fontsize=12,transform=ax.transAxes)
    
#     plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
#     plotFile= os.path.join(box_path, site+'_peak_demand_'+strategy+'.jpg')
#     plt.savefig(plotFile,dpi=1000)
#     plt.close()
    
    
# # ONE AY (1a)
# # repeat for average of top 10 peaks
# # BoxPlot peak across all VBs or top 10 peaks vs kWh
# # --------------------------------------------------

# # Use seaborn - boxplot:
# sns.set(style="whitegrid")
# cmap = mpl.cm.tab10_r
# colours = [cmap(k) for k in [1,2,3,4,5,6,7]]  #[0,0.5,1,1.5,2,2.5,3 ]]
# colourp = sns.color_palette(palette=colours)


# for strategy in battery_strategies:    

#     # --------------------------------------
#     dfen = df[df['arrangement'].isin(['en', 'en_pv'])]
#     dfx=dfen.copy().loc[dfen['central_battery_strategy']==strategy].append(
#         dfen.copy().loc[dfen['kWh_unit']==0]).append(
#         df[df['arrangement'] =='bau'])
#     # Create df for boxplots:
#     dfo = pd.DataFrame(columns=[ 'kWp_unit','kWh_unit', 'peak_demand'])

#     for s in dfx.index:
#         kWp = '{0:3.1f}'.format(dfx.loc[s,'kWp_unit']) + ' kWp / unit'
#         kWh = dfx.loc[s,'kWh_unit']
#         for vb in range (1,51):
#             short_label = dfx.loc[s,'short_label']
#             peak_demand = dfpeak10.loc[short_label, str(vb).zfill(2)]
#             i = str(s) + '_' + str(vb)
#             dfo.loc[i]= [kWp, kWh, peak_demand]


#     # Sort by kWp
#     dfo['pv_cat']=pd.Categorical(dfo['kWp_unit'],
#                               categories = ['0.0 kWp / unit',
#                                             '0.5 kWp / unit',
#                                             '1.0 kWp / unit',
#                                             '1.5 kWp / unit',
#                                             '2.0 kWp / unit',
#                                             '2.5 kWp / unit',
#                                             '3.0 kWp / unit'],                                         
#                               ordered=True)
#     dfo.sort_values(['pv_cat'], inplace=True)


#     # Plot Boxplot:
#     ax = sns.boxplot(data=dfo, x = 'kWh_unit', y= 'peak_demand', hue ='kWp_unit', 
#                      dodge=True, linewidth = 1, width=0.7,
#                     fliersize =3, notch = False,
#                     palette = colourp)
#     ax.set_xlabel("Battery Capacity (kWh / unit)", fontsize=14)
#     ax.set_ylabel("Peak Demand (kWh / 30 minutes)", fontsize=14)
#     ax.set_title('Mean of 10 demand peaks', fontsize=20, y=1.02)
#     ax.set_ylim(15,30)
#     #Legends outside box and resize chart to fit
#     box = ax.get_position()
#     ax.set_position([box.x0, box.y0, box.width * 0.7, box.height])
#     plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
#     ax.text(1.05,0.3, site_tag[site], fontsize=14,transform=ax.transAxes)
#     ax.text(0.1,0.1, strategy, fontsize=12,transform=ax.transAxes)
    
#     plotFile= os.path.join(box_path, site+'_peak10_demand_'+strategy+'.jpg')
#     plt.savefig(plotFile,dpi=1000)
#     plt.close()


In [12]:
# # ONE AY (1a)
# # BoxPlot peak across all VBs or top 10 peaks vs kWh
# # --------------------------------------------------
# for strategy in df['central_battery_strategy'].dropna().values[0]:
    
    
#     # --------------------------------------
#     dfen = df[df['arrangement'].isin(['en', 'en_pv'])]
#     dfx=dfen.copy().loc[dfen['central_battery_strategy']==strategy].append(
#         dfen.copy().loc[dfen['kWh_unit']==0])
#     # ----------------------
#     # reindex and stack data
#     # ----------------------
#     dfx.index = [dfx.kWh_unit,dfx.kWp_unit]
#     dfu = dfx['mean_peak'].unstack()
#     cols =dfu.columns
#     dfu.columns=cols
#     dfv=dfu.transpose()
#     dfv.loc[0,0]=df.loc[df.arrangement =='bau','mean_peak'].values[0]

#     # Use seaborn - boxplot:
#     sns.set(style="whitegrid")
#     cmap = mpl.cm.tab10_r
#     colours = [cmap(sites.index(site)) for site in [ 'G', 'H', 'I',  'J' ]]
#     colourp = sns.color_palette(palette=colours)

#     #dfx = dfo[dfo['site'].isin(['G','H','I','J'])]


# #     dfx['site_tag'] = dfx['site'].apply(lambda x : dict_tags[x])
    

   

#     # -----------------
#     # Plot Peak Demand vs kWh 
#     # -----------------
#     dfplot = dfv.copy().T
#     cols = dfplot.columns
#     cols2 = [str(round(c,1)) + ' kWp / unit' for c in cols]
#     dfplot.columns = cols2
#     fig, ax = plt.subplots()
#     # Legend outside box and resize chart to fit
#     box = ax.get_position()
#     ax.set_position([box.x0, box.y0, box.width * 0.7, box.height])
#     dfplot.plot(ax=ax,kind='line',grid=True, marker = 'o').legend(loc = 'upper left',bbox_to_anchor=(1.02, 1))
#     # maxy=50
#     # ax.set_ylim(0,55)
#     ax.set_ylabel('Mean Peak Demand (kWh / 30 mins)')
#     ax.set_xlabel('Storage kWh / unit')

#     # Label Parameters
#     # ----------------
#     pa = arrangement
#     pt = site_tag[site]
#     ps = strategy
#     ax.text(1.04,0.4,pa,transform=ax.transAxes,fontsize=fontsize)
#     ax.text(1.04,0.25,pt,transform=ax.transAxes,fontsize=fontsize)
#     ax.text(1.04,0.2,ps,transform=ax.transAxes,fontsize=fontsize)

#     pname = 'peak_kWh'  +'_'+ 'site_' + site + '_'+ strategy +'.jpg'
#     plotFile = os.path.join(plot_pd,pname)
#     plt.savefig(plotFile,dpi=1000)
#     plt.close()

  

In [21]:
sitelist=['F','G','H','I','J']

for study_base in ['bat_energy3_','bat_energy2_']:
    for sitename in sitelist:             
        # -------------------------------
        # combine  input and result files
        # -------------------------------

        project='EN2_bat2'
        study_name=study_base + sitename
        base_path='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_4\\studies'
        op = opm.Output(base_path = base_path,
                                  project = project,
                                  study_name = study_name)
        self = op
        df = self.data.copy()
        df_in = self.study_parameters.copy()
        df = df.merge(df_in,left_index = True, right_index=True)
        for c in df.columns:
            if '_x' in c:
                nc = c[:-2]
                df.rename(columns={c:nc}, inplace=True)
            if '_y' in c:
                df= df.drop([c],axis=1)
        df.head()
        arrangement = 'en_pv'
        fontsize=12
        num_units = df['number_of_households'][0]
        # get pv sizes for maximum pv systems
        # -----------------------------------
        maximums={}
        pv_ref_file='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_3\\reference\\capex_pv_lookup.csv'
        pv_ref=pd.read_csv(pv_ref_file)
        pv_ref = pv_ref.set_index('pv_cap_id')
        for i in pv_ref.index:
            if 'max' in i and 'site' in i:
                site = um.find_between(i,'_','_')
                maximums[site] = pv_ref.loc[i,'kW']

        # get kWp / unit ,kWh / unit  and % bau for each scenario
        # ------------------------------------------------------------
        df['pv_filename'].fillna('zero',inplace=True)

        for s in df.index:
            site = df.loc[s,'load_folder'][-1]
            df.loc[s,'site']=site
            if df.loc[s,'pv_filename'] == 'zero':
                df.loc[s,'kWp_unit'] =0
            elif 'max' in df.loc[s,'pv_filename']:
                df.loc[s,'kWp_unit']= maximums[site]/df.loc[s,'number_of_households']
            else:
                df.loc[s,'kWp_unit'] = float(df.loc[s,'pv_filename'][-9]) + float(df.loc[s,'pv_filename'][-7])/10

            df.loc[s,'kWh_unit']= df.fillna(0).loc[s,'central_battery_capacity_kWh']/df.loc[s,'number_of_households']
            df.loc[s,'short_label']= df.loc[s,'scenario_label'][14:24]


        # paths for peak demand files  
        # ---------------------------
        process_path = os.path.join(base_path,project,'outputs',study_name,'process')
        pf = 'peak_demand.csv'
        peakFile = os.path.join(process_path, pf)
        pf10 = 'peak_demand_10.csv'
        peakFile10 = os.path.join(process_path, pf10)

        #  reload files here if saved previously:
        dfpeak = pd.read_csv(peakFile)
        dfpeak=dfpeak.set_index('short_label')
        dfpeak10 = pd.read_csv(peakFile10)
        dfpeak10=dfpeak10.set_index('short_label')

        df['mean_peak']= df['short_label'].apply(lambda x:dfpeak.loc[x].mean())
        df['mean_peak_10highest']= df['short_label'].apply(lambda x:dfpeak10.loc[x].mean())
        df['mean_peak_pu']= df['mean_peak']/num_units


        # Directories for plots
        # ---------------------
        plot_pd = os.path.join(op.plot_path,'peak_demand')
        if not os.path.exists(plot_pd):
            os.makedirs(plot_pd)
        plot_pd10 = os.path.join(op.plot_path,'peak_demand_10highest')
        if not os.path.exists(plot_pd10):
            os.makedirs(plot_pd10)

        plot_pf = os.path.join(op.plot_path,'peak_frequency')
        if not os.path.exists(plot_pf):
            os.makedirs(plot_pf)

        box_path = os.path.join(op.plot_path,'peak_box')
        if not os.path.exists(box_path):
                os.makedirs(box_path)

        # Need to loop this for sites, if required

        # ONE
        # Plot average of peak across all VBs or average of top 10 peaks vs kWh & kWp
        # ---------------------------------------------------------------------------

        for strategy in df['central_battery_strategy'].drop_duplicates().dropna().values:

            # --------------------------------------
            dfen = df[df['arrangement'].isin(['en', 'en_pv'])]
            dfx=dfen.copy().loc[dfen['central_battery_strategy']==strategy].append(
                dfen.copy().loc[dfen['kWh_unit']==0])
            # ----------------------
            # reindex and stack data
            # ----------------------
            dfx.index = [dfx.kWh_unit,dfx.kWp_unit]
            dfu = dfx['mean_peak_pu'].unstack()
            cols =dfu.columns
            dfu.columns=cols
            dfv=dfu.transpose()
            dfv.loc[0,0]=df.loc[df.arrangement =='bau','mean_peak_pu'].values[0]


            # ---------------------
            # Plot peak Demand vs kW peak
            # ---------------------
            dfplot = dfv.copy()
            cols = dfplot.columns
            cols2 = [str(round(c,1)) + ' kWh / unit' for c in cols]
            dfplot.columns = cols2
            fig, ax = plt.subplots()
            # Legend outside box and resize chart to fit
            box = ax.get_position()
            ax.set_position([box.x0, box.y0, box.width * 0.7, box.height])
            dfplot.plot(ax=ax,kind='line',grid=True, marker = 'o').legend(loc = 'upper left',bbox_to_anchor=(1.02, 1))
            # maxy=50
            ax.set_ylim(yrange[sitename][0],yrange[sitename][1])
            ax.set_ylabel('Mean Peak Demand (kWh /unit /30 mins)')
            ax.set_xlabel('PV kWp / unit')

            # Label Parameters
            # ----------------
            pa = arrangement
            pt = site_tag[site]
            ps = strategy
            ax.text(1.04,0.4,pa,transform=ax.transAxes,fontsize=fontsize)
            ax.text(1.04,0.25,pt,transform=ax.transAxes,fontsize=fontsize)
            ax.text(1.04,0.2,ps,transform=ax.transAxes,fontsize=fontsize)

            pname = 'peak_pu_kWp'  +'_'+ 'site_'  +site + '_'+ strategy +'.jpg'
            plotFile = os.path.join(plot_pd,pname)
            plt.savefig(plotFile,dpi=1000)
            plt.close()

            # -----------------
            # Plot Peak Demand vs kWh 
            # -----------------
            dfplot = dfv.copy().T
            cols = dfplot.columns
            cols2 = [str(round(c,1)) + ' kWp / unit' for c in cols]
            dfplot.columns = cols2
            fig, ax = plt.subplots()
            # Legend outside box and resize chart to fit
            box = ax.get_position()
            ax.set_position([box.x0, box.y0, box.width * 0.7, box.height])
            dfplot.plot(ax=ax,kind='line',grid=True, marker = 'o').legend(loc = 'upper left',bbox_to_anchor=(1.02, 1))
            # maxy=50
            ax.set_ylim(yrange[sitename][0],yrange[sitename][1])
            ax.set_ylabel('Mean Peak Demand (kWh /unit /30 mins)')
            ax.set_xlabel('Storage kWh / unit')

            # Label Parameters
            # ----------------
            pa = arrangement
            pt = site_tag[site]
            ps = strategy
            ax.text(1.04,0.4,pa,transform=ax.transAxes,fontsize=fontsize)
            ax.text(1.04,0.25,pt,transform=ax.transAxes,fontsize=fontsize)
            ax.text(1.04,0.2,ps,transform=ax.transAxes,fontsize=fontsize)

            pname = 'peak_pu_kWh'  +'_'+ 'site_' + site + '_'+ strategy +'.jpg'
            plotFile = os.path.join(plot_pd,pname)
            plt.savefig(plotFile,dpi=1000)
            plt.close()

            # ------------------------------
            # Plot 3D Peak demand vs kWp and kWh 
            # ------------------------------
            dfplot = dfv.copy()
            x = dfplot.columns
            y = dfplot.index
            X,Y = np.meshgrid(x,y)
            Z = dfplot
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            ax.plot_surface(X, Y, Z)

            # Set Axes limits
            # ---------------
            ax.set_zlim(yrange[sitename][0],yrange[sitename][1])
            # ax.set_zlim(0,50)
            # maxz=50
            # ax.set_ylim(0,3.2)  # kWp
            # ax.set_xlim(0,4.2)  # kWh
            # ax.set_xticks([0,1,2,3])
            # ax.set_yticks([0,1,2,3,4])

            #Set rotation angle 
            #------------------
            ax.view_init(azim=30)
            #ax.view_init(elev=60)

            # Label Axes
            # ----------
            ax.set_zlabel('Mean Peak Demand (kWh / unit /30 mins)', fontsize=fontsize)
            ax.set_ylabel('PV kWp / unit',fontsize=fontsize)
            ax.set_xlabel('Storage kWh / unit',fontsize=fontsize)

            # Set Tick Lablels
            # ----------------
            ax.xaxis.set_major_locator(plt.MaxNLocator(5))
            ax.yaxis.set_major_locator(plt.MaxNLocator(4))

            # Label Parameters
            # ----------------
            pa = arrangement
            pt = site_tag[site]
            ps = strategy
            ax.text2D(0,1.07,pa,transform=ax.transAxes,fontsize=fontsize)
            ax.text2D(0,1.0,pt,transform=ax.transAxes,fontsize=fontsize)
            ax.text2D(0,0.93,ps,transform=ax.transAxes,fontsize=fontsize)

            pname = 'peak_pu_3D'  +'_'+ 'site_'  +site + '_'+ strategy + '_' + arrangement +'.jpg'
            plotFile = os.path.join(plot_pd,pname)
            plt.savefig(plotFile,dpi=1000)
            plt.close()

#         ## REPEAT FOR mean of 10 highest peaks:        
#         for strategy in df['central_battery_strategy'].drop_duplicates().dropna().values:

#             # --------------------------------------
#             # --------------------------------------
#             dfen = df[df['arrangement'].isin(['en', 'en_pv'])]
#             dfx=dfen.copy().loc[dfen['central_battery_strategy']==strategy].append(
#                 dfen.copy().loc[dfen['kWh_unit']==0])
#             # ----------------------
#             # reindex and stack data
#             # ----------------------
#             dfx.index = [dfx.kWh_unit,dfx.kWp_unit]
#             dfu = dfx['mean_peak_10highest'].unstack()
#             cols =dfu.columns
#             dfu.columns=cols
#             dfv=dfu.transpose()
#             dfv.loc[0,0]=df.loc[df.arrangement =='bau','mean_peak_10highest'].values[0]

#             # ---------------------
#             # Plot peak Demand vs kW peak
#             # ---------------------
#             dfplot = dfv.copy()
#             cols = dfplot.columns
#             cols2 = [str(round(c,1)) + ' kWh / unit' for c in cols]
#             dfplot.columns = cols2
#             fig, ax = plt.subplots()
#             # Legend outside box and resize chart to fit
#             box = ax.get_position()
#             ax.set_position([box.x0, box.y0, box.width * 0.7, box.height])
#             dfplot.plot(ax=ax,kind='line',grid=True, marker = 'o').legend(loc = 'upper left',bbox_to_anchor=(1.02, 1))
#             # maxy=50
#             # ax.set_ylim(0,55)
#             ax.set_ylabel('Mean Peak Demand (10 highest) (kWh / 30 mins)')
#             ax.set_xlabel('PV kWp / unit')

#             # Label Parameters
#             # ----------------
#             pa = arrangement
#             pt = site_tag[site]
#             ps = strategy
#             ax.text(1.04,0.4,pa,transform=ax.transAxes,fontsize=fontsize)
#             ax.text(1.04,0.25,pt,transform=ax.transAxes,fontsize=fontsize)
#             ax.text(1.04,0.2,ps,transform=ax.transAxes,fontsize=fontsize)

#             pname = 'peak_kWp'  +'_'+ 'site_'  +site + '_'+ strategy +'.jpg'
#             plotFile = os.path.join(plot_pd10,pname)
#             plt.savefig(plotFile,dpi=1000)
#             plt.close()

#             # -----------------
#             # Plot Peak Demand vs kWh 
#             # -----------------
#             dfplot = dfv.copy().T
#             cols = dfplot.columns
#             cols2 = [str(round(c,1)) + ' kWp / unit' for c in cols]
#             dfplot.columns = cols2
#             fig, ax = plt.subplots()
#             # Legend outside box and resize chart to fit
#             box = ax.get_position()
#             ax.set_position([box.x0, box.y0, box.width * 0.7, box.height])
#             dfplot.plot(ax=ax,kind='line',grid=True, marker = 'o').legend(loc = 'upper left',bbox_to_anchor=(1.02, 1))
#             # maxy=50
#             # ax.set_ylim(0,55)
#             ax.set_ylabel('Mean Peak Demand (10 highest) (kWh / 30 mins)')
#             ax.set_xlabel('Storage kWh / unit')

#             # Label Parameters
#             # ----------------
#             pa = arrangement
#             pt = site_tag[site]
#             ps = strategy
#             ax.text(1.04,0.4,pa,transform=ax.transAxes,fontsize=fontsize)
#             ax.text(1.04,0.25,pt,transform=ax.transAxes,fontsize=fontsize)
#             ax.text(1.04,0.2,ps,transform=ax.transAxes,fontsize=fontsize)

#             pname = 'peak_kWh'  +'_'+ 'site_' + site + '_'+ strategy +'.jpg'
#             plotFile = os.path.join(plot_pd10,pname)
#             plt.savefig(plotFile,dpi=1000)
#             plt.close()

#             # ------------------------------
#             # Plot 3D Peak Demand vs kWp and kWh 
#             # ------------------------------
#             dfplot = dfv.copy()
#             x = dfplot.columns
#             y = dfplot.index
#             X,Y = np.meshgrid(x,y)
#             Z = dfplot
#             fig = plt.figure()
#             ax = fig.add_subplot(111, projection='3d')
#             ax.plot_surface(X, Y, Z)

#             # Set Axes limits
#             # ---------------
#             # ax.set_zlim(0,50)
#             # maxz=50
#             # ax.set_ylim(0,3.2)  # kWp
#             # ax.set_xlim(0,4.2)  # kWh
#             # ax.set_xticks([0,1,2,3])
#             # ax.set_yticks([0,1,2,3,4])

#             #Set rotation angle 
#             #------------------
#             ax.view_init(azim=30)
#             #ax.view_init(elev=60)

#             # Label Axes
#             # ----------
#             ax.set_zlabel('Mean Peak Demand (10 highest) (kWh / 30 mins)', fontsize=fontsize)
#             ax.set_ylabel('PV kWp / unit',fontsize=fontsize)
#             ax.set_xlabel('Storage kWh / unit',fontsize=fontsize)

#             # Set Tick Lablels
#             # ----------------
#             ax.xaxis.set_major_locator(plt.MaxNLocator(5))
#             ax.yaxis.set_major_locator(plt.MaxNLocator(4))

#             # Label Parameters
#             # ----------------
#             pa = arrangement
#             pt = site_tag[site]
#             ps = strategy
#             ax.text2D(0,1.07,pa,transform=ax.transAxes,fontsize=fontsize)
#             ax.text2D(0,1.0,pt,transform=ax.transAxes,fontsize=fontsize)
#             ax.text2D(0,0.93,ps,transform=ax.transAxes,fontsize=fontsize)

#             pname = 'peak_3D'  +'_'+ 'site_'  +site + '_'+ strategy + '_' + arrangement +'.jpg'
#             plotFile = os.path.join(plot_pd10,pname)
#             plt.savefig(plotFile,dpi=1000)
#             plt.close()


44.0